In [1]:
%load_ext autoreload
%autoreload 2

# Test updated version of PCR pipe

In [2]:
import copy

from pathlib import Path
from time import time

import faiss
import numpy as np
import open3d as o3d
import open3d.core as o3c
import pandas as pd
import torch

try:
    import torchshow as ts
except ImportError:
    ts = None
    print("torchshow not installed, skipping visualization")

from hydra.utils import instantiate
from omegaconf import OmegaConf
from scipy.spatial.transform import Rotation
from torch import Tensor
from torch.utils.data import DataLoader
from tqdm import tqdm

from geotransformer.utils.registration import compute_registration_error


from opr.datasets import NCLTDataset
from opr.pipelines.place_recognition import PlaceRecognitionPipeline
from opr.pipelines.registration.pointcloud import PointcloudRegistrationPipeline
from opr.pipelines.localization import LocalizationPipeline

torchshow not installed, skipping visualization


In [3]:
import os

os.environ["DISPLAY"] = ":1"

torch.cuda.synchronize()

#### Weights download

##### Place Recognition

You can download the `multi-image_lidar_late-fusion_nclt.pth` from the HuggingFace model hub:
https://huggingface.co/OPR-Project/PlaceRecognition-NCLT

```bash
wget https://huggingface.co/OPR-Project/PlaceRecognition-NCLT/resolve/main/multi-image_lidar_late-fusion_nclt.pth
```

##### Registration

You can download the `hregnet_nuscenes.pth` from the HuggingFace model hub:
https://huggingface.co/OPR-Project/Registration-nuScenes.

```bash
wget https://huggingface.co/OPR-Project/Registration-nuScenes/resolve/main/hregnet_nuscenes.pth
```

In [4]:
DATASET_ROOT = "/home/docker_opr/Datasets/OpenPlaceRecognition/NCLT_preprocessed"

SENSOR_SUITE = ["image_Cam5", "image_Cam2", "pointcloud_lidar"]

BATCH_SIZE = 32
NUM_WORKERS = 4
DEVICE = "cuda"

PR_MODEL_CONFIG_PATH = "../configs/model/place_recognition/multi-image_lidar_late-fusion.yaml"
PR_WEIGHTS_PATH = "../weights/place_recognition/multi-image_lidar_late-fusion_nclt.pth"

REGISTRATION_MODEL_CONFIG_PATH = "../configs/model/registration/hregnet.yaml"
REGISTRATION_WEIGHTS_PATH = "../weights/registration/hregnet_nuscenes.pth"

In [5]:
TRACK_LIST = sorted([str(subdir.name) for subdir in Path(DATASET_ROOT).iterdir() if subdir.is_dir()])
print(f"Found {len(TRACK_LIST)} tracks")
print(TRACK_LIST)

print("WARNING: track list limited")
TRACK_LIST = TRACK_LIST[:2]
print(TRACK_LIST)

Found 10 tracks
['2012-01-08', '2012-01-22', '2012-02-12', '2012-02-18', '2012-03-31', '2012-05-26', '2012-08-04', '2012-10-28', '2012-11-04', '2012-12-01']
['2012-01-08', '2012-01-22']


In [6]:
pr_model_config = OmegaConf.load(PR_MODEL_CONFIG_PATH)
pr_model = instantiate(pr_model_config)
pr_model.load_state_dict(torch.load(PR_WEIGHTS_PATH))
pr_model = pr_model.to(DEVICE)
pr_model.eval();

reg_model_config = OmegaConf.load(REGISTRATION_MODEL_CONFIG_PATH)
reg_model = instantiate(reg_model_config)
reg_model.load_state_dict(torch.load(REGISTRATION_WEIGHTS_PATH))
reg_model = reg_model.to(DEVICE)
reg_model.eval();

2025-03-24 19:23:00.012 | WARNING  | opr.models.place_recognition.pointmamba:<module>:16 - The 'pointmamba' package is not installed. Please install it manually if neccessary.
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/docker_opr/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:05<00:00, 8.49MB/s]


In [7]:
!rm -rf /home/docker_opr/Datasets/OpenPlaceRecognition/NCLT_preprocessed/2012-01-08/HRegNet_features

In [8]:
pr_pipe = PlaceRecognitionPipeline(
    database_dir=Path(DATASET_ROOT) / TRACK_LIST[0],
    model=pr_model,
    model_weights_path=PR_WEIGHTS_PATH,
    device=DEVICE,
)
reg_pipe = PointcloudRegistrationPipeline(
    model=reg_model,
    model_weights_path=REGISTRATION_WEIGHTS_PATH,
    device=DEVICE,
    voxel_downsample_size=0.3,
    num_points_downsample=8192,
)
loc_pipe = LocalizationPipeline(
    place_recognition_pipeline=pr_pipe,
    registration_pipeline=reg_pipe,
    precomputed_reg_feats=True,
    pointclouds_subdir="velodyne_data",
)

2025-03-24 19:23:06.376 | WARNING  | opr.pipelines.localization.base:_setup_precomputed_reg_feats:95 - Precomputed registration features directory not found: /home/docker_opr/Datasets/OpenPlaceRecognition/NCLT_preprocessed/2012-01-08/HRegNet_features. It will be created and features will be computed.
2025-03-24 19:23:06.376 | WARNING  | opr.pipelines.localization.base:_setup_precomputed_reg_feats:105 - Precomputed registration features directory is empty. Computing features.
  0%|          | 0/1261 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/hregnet/utils.py:24: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  output = torch.cuda.IntTensor(B, npoint)


In [9]:
query_dataset = NCLTDataset(
    dataset_root=DATASET_ROOT,
    subset="test",
    data_to_load=SENSOR_SUITE,
    pointcloud_quantization_size=0.5,
    max_point_distance=None,
)
query_dataset.dataset_df = query_dataset.dataset_df[query_dataset.dataset_df["track"] == TRACK_LIST[1]]
query_dataset.dataset_df.reset_index(inplace=True)

query_df = pd.read_csv(Path(DATASET_ROOT) / TRACK_LIST[1] / "track.csv", index_col=0)
query_df = query_df[query_df['image'].isin(query_dataset.dataset_df['image'])].reset_index(drop=True)

In [10]:
times = []
for query in tqdm(query_dataset):
    t = time()
    output = loc_pipe.infer(query)
    times.append(time() - t)
times = np.array(times[1:])

  0%|          | 0/275 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
100%|██████████| 275/275 [00:24<00:00, 11.45it/s]


In [11]:
print(loc_pipe.reg_pipe.stats_history.keys())
print(f"{np.mean(loc_pipe.reg_pipe.stats_history['total_time'][1:]) * 1000:.2f} ms")

dict_keys(['inference_time', 'downsample_time', 'total_time'])
54.97 ms


## Compare downsample performance

In [12]:
cpu_times = []
for query in tqdm(query_dataset, desc="CPU", leave=False):
    query_pc = query["pointcloud_lidar_coords"].to("cpu")
    t_s = time()
    query_pc_downsampled = reg_pipe._downsample_pointcloud(query_pc)
    cpu_times.append(time() - t_s)

gpu_times = []
for query in tqdm(query_dataset, desc="GPU", leave=False):
    query_pc = query["pointcloud_lidar_coords"].to("cuda")
    t_s = time()
    query_pc_downsampled = reg_pipe._downsample_pointcloud(query_pc)
    gpu_times.append(time() - t_s)

print(f"CPU:  {np.mean(cpu_times) * 1000:.2f} ms")
print(f"GPU:  {np.mean(gpu_times) * 1000:.2f} ms")

CPU:   0%|          | 0/275 [00:00<?, ?it/s]

CPU:  11.77 ms
GPU:  0.85 ms
